In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,0,28,2024-07-01 20:27:19
1,大安區,21,0,21,2024-07-01 20:21:14
2,大安區,16,6,10,2024-07-01 20:26:19
3,大安區,11,0,11,2024-07-01 20:19:14
4,大安區,16,3,13,2024-07-01 20:27:19
...,...,...,...,...,...
1424,臺大公館校區,30,1,28,2024-07-01 20:26:19
1425,臺大公館校區,20,2,18,2024-07-01 20:27:19
1426,臺大公館校區,24,2,21,2024-07-01 20:25:23
1427,臺大公館校區,40,30,1,2024-07-01 20:27:19


In [2]:
bins = [0,1,4,7,11,100]
labels = ['0輛','1~3輛','4~6輛','7~10輛','10輛以上']

df['可借'] = pd.cut(df['可借數量'],bins=bins,right=False,labels=labels)
df

,行政區域,總數量,可借數量,可還數量,時間,可借
0,大安區,28,0,28,2024-07-01 20:27:19,0輛
1,大安區,21,0,21,2024-07-01 20:21:14,0輛
2,大安區,16,6,10,2024-07-01 20:26:19,4~6輛
3,大安區,11,0,11,2024-07-01 20:19:14,0輛
4,大安區,16,3,13,2024-07-01 20:27:19,1~3輛
...,...,...,...,...,...,...
1424,臺大公館校區,30,1,28,2024-07-01 20:26:19,1~3輛
1425,臺大公館校區,20,2,18,2024-07-01 20:27:19,1~3輛
1426,臺大公館校區,24,2,21,2024-07-01 20:25:23,1~3輛
1427,臺大公館校區,40,30,1,2024-07-01 20:27:19,10輛以上
